# Digits dataset

#### Loading the libraries

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.datasets import load_digits
from sklearn.preprocessing import normalize
import random
import matplotlib.pyplot as plt
import auxiliary_fun as a
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import time
from keras import utils
from keras.models import Sequential
from keras.layers import Dense, Activation
import random
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification as mk
from keras.metrics import Precision, Recall, AUC, MeanSquaredError
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam,SGD
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation

## Load dataset

In [ ]:
digits = load_digits()
Xor = normalize(digits.data)
yor = digits.target
print(len(Xor))

### Obtaining the dominating dataset with $\varepsilon \le 0.2$

In [ ]:
start = time.time()
index = a.dominatingSet(Xor,yor,epsilon=0.2)
Xsub=Xor[index]
ysub=yor[index]
end = time.time()
print("Dominating dataset, time for computation: ",end - start)
print("Dominating set size: ",np.shape(Xsub))

### Training a neural network and evaluating on the original dataset

In [ ]:
epochs = 1000

In [ ]:
cat_yor = utils.to_categorical(yor,10)
cat_ysub = utils.to_categorical(ysub,10)

In [ ]:
l = []
for i in range(5):
    model1 = Sequential()
    model1.add(Dense(units=400, activation='sigmoid', input_shape=(64,)))
    model1.add(Dense(units=800, activation='sigmoid'))
    model1.add(Dense(units=300, activation='sigmoid'))
    model1.add(Dense(units=800, activation='sigmoid'))
    model1.add(Dense(units=300, activation='sigmoid'))
    model1.add(Dense(units=10, activation='softmax'))
    model1.compile(optimizer="Adam", 
                   loss='categorical_crossentropy', 
                   metrics=['accuracy',
                            Recall(), 
                            Precision(),
                            AUC(),
                            MeanSquaredError()])
    start = time.time()
    history1=model1.fit(Xor, cat_yor,batch_size=len(Xor), 
                                #batch_size=1,
                                #validation_split = val_split, 
                                epochs=epochs,
                                verbose=False)
    end = time.time()
    print(abs(start-end))
    l.append(model1.evaluate(Xor,cat_yor,verbose=0))
l = np.array(l)
print(l)
plt.plot(history1.history['accuracy'])

In [ ]:
l2 = []
for i in range(5):
    model2 = Sequential()
    model2.add(Dense(units=400, activation='sigmoid', input_shape=(64,)))
    model2.add(Dense(units=800, activation='sigmoid'))
    model2.add(Dense(units=300, activation='sigmoid'))
    model2.add(Dense(units=800, activation='sigmoid'))
    model2.add(Dense(units=300, activation='sigmoid'))
    model2.add(Dense(units=10, activation='softmax'))
    model2.compile(optimizer="Adam", 
                   loss='categorical_crossentropy', 
                   metrics=['accuracy',
                            Recall(), 
                            Precision(),
                            AUC(),
                            MeanSquaredError()])
    import time
    start = time.time()
    history2=model2.fit(Xsub, cat_ysub,batch_size=len(Xsub),
                                #batch_size=1,
                                #validation_split = val_split, 
                                epochs=epochs,
                                verbose=False)
    end = time.time()
    print(abs(start-end))
    l2.append(model2.evaluate(Xor,cat_yor,verbose=0))
l2=np.array(l2)
print(l2)
plt.plot(history2.history['accuracy'])

### Random dataset

In [ ]:
X=np.column_stack((Xor,yor))
np.random.shuffle(X)
Xor = X[:,0:64]
yor = X[:,64]
cat_yor = utils.to_categorical(yor,10)
random_index = random.sample(range(len(Xor)),len(Xsub))
Xrand = Xor[random_index]
yrand = yor[random_index]
cat_yrand = utils.to_categorical(yrand,10)
digits_random=np.column_stack((Xrand,yrand))
#np.savetxt("digits_random3.txt",digits_random)

In [ ]:
#digits_random=np.column_stack((Xrand,yrand))
#np.savetxt("digits_random.txt",digits_random)
digits_random=np.loadtxt("data/digits_random.txt")
Xrand = digits_random[:,0:64]
yrand = digits_random[:,64]
cat_yrand = utils.to_categorical(yrand,10)

In [ ]:
l3 = []
for i in range(5):
    model3 = Sequential()
    model3.add(Dense(units=400, activation='sigmoid', input_shape=(64,)))
    model3.add(Dense(units=800, activation='sigmoid'))
    model3.add(Dense(units=300, activation='sigmoid'))
    model3.add(Dense(units=800, activation='sigmoid'))
    model3.add(Dense(units=300, activation='sigmoid'))
    model3.add(Dense(units=10, activation='softmax'))
    model3.compile(optimizer="Adam", 
                   loss='categorical_crossentropy', 
                   metrics=['accuracy',
                            Recall(), 
                            Precision(),
                            AUC(),
                            MeanSquaredError()])
    history3=model3.fit(Xrand, cat_yrand, 
                        batch_size=len(Xrand), 
                        epochs= epochs, 
                        verbose=False)
    l3.append(model3.evaluate(Xor,cat_yor,verbose=0))
l3 = np.array(l3)
print(l3)
plt.plot(history3.history['accuracy'])

In [ ]:
np.mean(l3[:,1])

### Hausdorff and Bottleneck distance

In [ ]:
from scipy.spatial.distance import directed_hausdorff
print("Hausdorff distance between the original dataset and the dominating dataset: ",max(directed_hausdorff(Xor, Xsub)[0], directed_hausdorff(Xsub, Xor)[0])/2)
print("Hausdorff distance between the original dataset and the random dataset: ",max(directed_hausdorff(Xor, Xrand)[0], directed_hausdorff(Xrand, Xor)[0])/2)

In [ ]:
from ripser import ripser
diagrams_or = ripser(Xor,maxdim=2)['dgms']
diagrams_Sub = ripser(Xsub,maxdim=2)['dgms']
diagrams_Rand = ripser(Xrand,maxdim=2)['dgms']

In [ ]:
import gudhi as g
message = "Bottleneck distance for dominating dataset and dimension 0 =" + '%.2f' % g.bottleneck_distance(diagrams_or[0], diagrams_Sub[0])
print(message)
message = "Bottleneck distance for dominating dataset and dimension 1 =" + '%.2f' % g.bottleneck_distance(diagrams_or[1], diagrams_Sub[1])
print(message)
message = "Bottleneck distance for Random dataset and dimension 0 =" + '%.2f' % g.bottleneck_distance(diagrams_or[0], diagrams_Rand[0])
print(message)
message = "Bottleneck distance for Random dataset and dimension 1 =" + '%.2f' % g.bottleneck_distance(diagrams_or[1], diagrams_Rand[1])
print(message)



In [ ]:
from gtda.homology import VietorisRipsPersistence

In [ ]:
homology_dimensions=[0,1,2]
persistence = VietorisRipsPersistence(
    metric = "euclidean",
    homology_dimensions = homology_dimensions,
)
diagrams_basic = persistence.fit_transform([Xor,Xsub,Xrand])

In [ ]:
from gtda.plotting import plot_diagram

In [ ]:
plot_diagram(diagrams_basic[2])